In [1]:
import pandas as pd
import numpy as np
import pickle

# Radius of Earth
R = 6371e3

columns_probes = ['sampleID', 'dateTime', 'sourceCode', 'latitude', 'longitude', 'altitude', 'speed', 'heading']
columns_links = ['linkPVID', 'refNodeID', 'nrefNodeID', 'length', 'functionalClass', 'directionOfTravel', 'speedCategory',
                 'fromRefSpeedLimit', 'toRefSpeedLimit', 'fromRefNumLanes', 'toRefNumLanes', 'multiDigitized', 'urban',
                 'timeZone', 'shapeInfo', 'curvatureInfo', 'slopeInfo']

df_probes = pd.read_pickle('probes_pickle.pkl')
df_links = pd.read_pickle('links_pickle.pkl')


In [2]:
# probe_coord = [[df_probes['latitude'][i], df_probes['longitude'][i]] for i in range(len(df_probes))]
# with open('probe_coord.pkl', 'wb') as f:
#     pickle.dump(probe_coord, f)

# Opening the pickled probes coordinates
with open('probe_coord.pkl', 'rb') as f:
    probe_coord = pickle.load(f)

In [3]:
# Distinguishing the reference point and non-reference point of all links in two lists

link_shape = df_links['shapeInfo'].str.split('|')
ref_coord = [link_shape[i][0].split('/') for i in range(len(link_shape))]
nonref_coord = [link_shape[i][-1].split('/') for i in range(len(link_shape))]

In [4]:
# Replacing the elevation coordinate with 0 if its not present

for i in range(len(ref_coord)):
    for j in range(3):
        if ref_coord[i][j] == '':
            ref_coord[i][j] = '0'
ref_coord = list(map(lambda sl: list(map(float, sl)), ref_coord))

for i in range(len(nonref_coord)):
    for j in range(3):
        if nonref_coord[i][j] == '':
            nonref_coord[i][j] = '0'
nonref_coord = list(map(lambda sl: list(map(float, sl)), nonref_coord))


In [5]:
# Functions to get Bearing between two points, distance of a point from a Great Circle Path given by two points (reference and non reference in this case), 
# and getting a list of great circle path distances for all the links from a certain point (probe coordinate).

def get_bearing(point_1, point_2):
    y = np.cos(point_2[0]) * np.sin(point_2[1] - point_1[1])
    x = np.cos(point_1[0]) * np.sin(point_2[0]) - np.sin(point_1[0]) * np.cos(point_2[0]) * np.cos(point_2[1] - point_1[1])
    return (np.degrees(np.arctan2(y, x)) + 360) % 360

def get_dist_from_path(point, ref, non_ref):
    ra_ref = np.radians(ref)
    ra_point = np.radians(point)

    angular_distance_13 = np.arccos(np.sin(ra_ref[0]) * np.sin(ra_point[0]) + np.cos(ra_ref[0]) * np.cos(ra_point[0]) * np.cos(abs(ra_point[1] - ra_ref[1])))
    theta_13 = get_bearing(ref, point)
    theta_12 = get_bearing(ref, non_ref)

    d = abs(np.arcsin(np.sin(angular_distance_13) * np.sin(theta_13 - theta_12)) * R)

    return d

def get_dist_list(point, ref, non_ref):
    dist_list = []
    for i in range(len(ref)):
        dist_list.append(get_dist_from_path(point, ref[i], non_ref[i]))
    return np.array(dist_list)

In [6]:
# Combining everything for probe matching for all probe points
N = 10

matched_links_indices = []
dist_from_ref = []
dist_from_link = []
p_index = 0

for sampleID in df_probes['sampleID'].unique()[:3]:

    # Getting distance from all links for 1 probe point which will be used to calculate closest N links from it
    dist_list = get_dist_list(probe_coord[p_index], ref_coord, nonref_coord)

    # Getting the index of the closest N distances for 1 probe point which will remain constant for a Sample ID since
    # probe points should not go really far away from their recent links
    fake_dist_list = dist_list.copy()
    closest_n_ind = []
    for i in range(N):
        closest_n_ind.append(fake_dist_list.argmin())
        fake_dist_list[closest_n_ind[i]] = np.inf

    print(sampleID, p_index)
    print(closest_n_ind)

    while (df_probes['sampleID'][p_index] == sampleID):

        # For each probe point of a sample ID use the heading and its distance from the N links calculated earlier
        # to create a selection criteria for the link
        headings = []
        updated_dist_list = []
        for close_index in closest_n_ind:
            ref = ref_coord[close_index]
            non_ref = nonref_coord[close_index]
            
            # Measuring point to great circle path distance
            updated_dist_list.append(get_dist_from_path(probe_coord[p_index], ref_coord[close_index], nonref_coord[close_index]))

            # Bearing Calculations from reference to non reference point on the link
            bearing = get_bearing(ref, non_ref)

            heading_factor = abs(df_probes['heading'][p_index] - bearing) / 720

            headings.append(heading_factor)

        headings = np.array(headings)
        
        updated_dist_list /= updated_dist_list[-1]
        # print(updated_dist_list)

        # Creating the selection list
        selection_list = (updated_dist_list + headings) / 2
        # print(selection_list)

        # Selected link's index
        selected_link_index = closest_n_ind[selection_list.argmin()]
        # print(selected_link_index)


        # Info for the new dataframe to be formed as instructed in the readme
        d = get_dist_from_path(probe_coord[p_index], ref_coord[selected_link_index], nonref_coord[selected_link_index])

        # Calculating distance from the reference node to the map-matched probe point location on the link in decimal meters
        ra_ref = np.radians(ref_coord[selected_link_index])
        ra_point = np.radians(probe_coord[p_index])
        angular_distance = np.arccos(np.sin(ra_ref[0]) * np.sin(ra_point[0]) + np.cos(ra_ref[0]) * np.cos(ra_point[0]) * np.cos(abs(ra_point[1] - ra_ref[1])))

        along_track_d = np.arccos(np.cos(angular_distance) / np.cos(d / R)) * R


        matched_links_indices.append(selected_link_index)
        dist_from_ref.append(along_track_d)
        dist_from_link.append(d)

        p_index += 1

# print(matched_links_indices)

3496 0
[103262, 24977, 92503, 178880, 95440, 1, 196340, 90559, 198177, 178894]
4552 63
[173013, 155414, 89441, 89442, 62636, 92172, 66451, 197343, 163363, 180702]
4553 125
[184949, 180252, 112240, 38487, 23807, 38485, 78004, 120, 120557, 69411]


In [7]:
# Creating the matched points csv as described in the readme

matched_points_df = df_probes.copy()[:len(matched_links_indices)]
matched_points_df['linkPVID'] = df_links['linkPVID'][matched_links_indices].reset_index(drop=True)
matched_points_df['direction'] = df_links['directionOfTravel'][matched_links_indices].reset_index(drop=True)
matched_points_df = matched_points_df.replace('B', 'F')
matched_points_df['distFromRef'] = dist_from_ref
matched_points_df['distFromLink'] = dist_from_link
# mathed_points_df.to_pickle('matched_points.pkl')
matched_points_df.to_csv('matched_points_1.csv', index=False, header=True)

In [ ]:
df_matched_points = pd.read_pickle('matched_points.pkl')
print(df_matched_points)